# Readers

> This module provides a unified interface for loading evaluation data from organizational repositories (IOM, UNHCR, etc.) and transforming it into standardized JSON format.

**Components:**

- `EvalReader`: Base interface for all repository readers
- `IOMRepoReader`: Reads IOM evaluation CSV exports from [evaluation.iom.int/evaluation-search-pdf](https://evaluation.iom.int/evaluation-search-pdf)
- `load_evals`: Loads processed evaluation JSON files

**Purpose:**

Transform diverse evaluation data sources into a consistent format for downstream analysis and processing.

In [ ]:
#| default_exp readers

In [ ]:
#| export
#| export
from pathlib import Path
from rich import print
from dataclasses import dataclass
import json
import hashlib
import pandas as pd
from fastcore.all import *

The `EvalReader` interface defines a common contract that all evaluation repository readers (e.g. IOM, UNHCR) must implement:

## Core Reader Interface

In [ ]:
#| exports
class EvalReader:
    def __init__(self, 
                 cfg:dict # Configuration dict with field mappings and processing rules
                ): store_attr()
    def read(self): raise NotImplementedError
    def tfm(self, df): raise NotImplementedError
    def to_json(self, output_path): raise NotImplementedError
    def __call__(self):
        df = self.read()
        return self.tfm(df)

In [ ]:
#| exports
def iom_input_cfg():
    return {
        'date_cols': ['Date of Publication', 'Evaluation Period From Date', 'Evaluation Period To Date'],
        'string_cols': ['Year'],
        'list_fields': {
            'Countries Covered': {'separator': ',', 'clean': True}
        },
        'document_fields': ['Document Subtype', 'File URL', 'File description'],
        'id_gen': {
            'method': 'md5',
            'fields': ['Title', 'Year', 'Project Code']  # fields to hash
        },
        'field_mappings': {
            'Title': 'title',
            'Year': 'year',
            # other mappings
        }
    }

In [ ]:
#| exports
@dataclass
class Evaluation:
    "An evaluation with rich notebook display"
    id:str
    docs:list
    meta:dict
        
    def _repr_markdown_(self):
        title = self.meta.get('Title', 'Untitled')
        year = self.meta.get('Year', 'n/a')
        org = self.meta.get('Evaluation Commissioner', 'Unknown')
        countries = self.meta.get('Countries Covered', [])
        country_str = ', '.join(countries[:3]) if countries else 'Not specified'
        if len(countries) > 3: country_str += f' (+{len(countries)-3} more)'
        
        return f"""
### {title}
**Year:** {year} | **Organization:** {org} | **Countries:** {country_str}

**Documents:** {len(self.docs)} available  
**ID:** `{self.id}`
"""

## IOM Reader

In [ ]:
#| exports
class IOMRepoReader(EvalReader):
    def __init__(self, 
                 fname:Path # Path to the CSV export file
                 ): 
        cfg = iom_input_cfg()  
        super().__init__(cfg)
        store_attr()


The `read` method loads the raw CSV data from the IOM repository export:

In [ ]:
#| exports
@patch
def read(self:IOMRepoReader): return pd.read_csv(self.fname)

In [ ]:
fname = 'files/test/evaluation-search-export-11_13_2025--18_09_44.csv'
reader = IOMRepoReader(fname)
reader.read().iloc[0]

Title                                       EVALUATION OF IOM’S MIGRATION DATA STRATEGY
Year                                                                               2025
Author                                                              Data Conscious Ltd.
Best Practicesor Lessons Learnt                                                     Yes
Date of Publication                                                          2025-08-11
Donor                                                                               NaN
Evaluation Brief                                                                    Yes
Evaluation Commissioner                                                             IOM
Evaluation Coverage                                                              Global
Evaluation Period From Date                                                  2025-08-20
Evaluation Period To Date                                                    2025-05-31
Executive Summary               

Each evaluation needs a unique identifier. Since the CSV doesn't include one, we generate an MD5 hash from key fields:

In [ ]:
#| exports
@patch
def _mk_id(self:IOMRepoReader, 
           row # DataFrame row containing evaluation metadata
          ):
    id_str = ''.join(str(row[f]) for f in self.cfg['id_gen']['fields'])
    return hashlib.md5(id_str.encode('utf-8')).hexdigest()

In [ ]:
reader = IOMRepoReader(fname)
df_test = reader.read()
eval_id = reader._mk_id(df_test.iloc[0])
test_eq(len(eval_id), 32)
test_eq(eval_id, '9992310969aa2f428bc8aba29f865cf3')

IOM evaluations can have multiple associated documents (reports, briefs, annexes). The CSV stores these as comma-separated values in three parallel fields. We parse and combine them into structured document records:

## Document Handling

In [ ]:
#| exports
@patch
def _mk_docs(self:IOMRepoReader, 
             row # DataFrame row with document fields
            ):
    "Parse document fields into structured records"
    stypes = [s.strip() for s in str(row['Document Subtype']).split(', ')]
    urls = [u.strip() for u in str(row['File URL']).split(', ')]
    descs = [d.strip() for d in str(row['File description']).split(', ')]
    return [dict(subtype=st, url=u, desc=d) for st,u,d in zip(stypes,urls,descs) if u.strip()]

In [ ]:
reader = IOMRepoReader(fname)
df_test = reader.read()
reader._mk_docs(df_test.iloc[0])

[{'subtype': 'Evaluation report',
  'url': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Annex%20VI%20Case%20Study%20-%20RDH%20East%2C%20Horn%20and%20Southern%20Africa.pdf',
  'desc': 'Evaluation Report'},
 {'subtype': 'Special related reports/documents',
  'url': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Annex%20VII%20Case%20Study%20-%20RDH%20Asia-Pacific.pdf',
  'desc': 'Evaluation Brief'}]

## Data Processing

In [ ]:
#| exports
@patch
def _proc_dates(self:IOMRepoReader, df):
    df[self.cfg['date_cols']] = df[self.cfg['date_cols']].astype(str)
    return df

In [ ]:
reader = IOMRepoReader(fname)
df_test = reader.read()
df_proc = reader._proc_dates(df_test)
test_eq(df_proc['Date of Publication'].dtype, 'object')
test_eq(df_proc['Evaluation Period From Date'].dtype, 'object')
test_eq(df_proc['Evaluation Period To Date'].dtype, 'object')
df_proc['Date of Publication'].iloc[0]

'2025-08-11'

In [ ]:
#| exports
@patch
def _proc_lists(self:IOMRepoReader, df):
    for fname,fcfg in self.cfg['list_fields'].items():
        df[fname] = df[fname].astype(str).str.split(fcfg['separator']).apply(lambda x: [item.strip() for item in x if item.strip()])
    return df

In [ ]:
reader = IOMRepoReader(fname)
df_test = reader.read()
df_proc = reader._proc_lists(df_test)
test_eq(type(df_proc['Countries Covered'].iloc[190]), list)
df_proc['Countries Covered'].iloc[190]

['Colombia']

In [ ]:
#| exports
@patch
def _to_dict(self:IOMRepoReader, row):
    "Convert row to evaluation dict"
    meta_cols = [col for col in row.index if col not in ['id', 'docs']]
    return dict(id=row['id'], docs=row['docs'], meta={f:row[f] for f in meta_cols})

In [ ]:
#| exports
@patch
def _to_eval(self:IOMRepoReader, row):
    "Convert row to Evaluation object"
    meta_cols = [col for col in row.index if col not in ['id', 'docs']]
    return Evaluation(id=row['id'], docs=row['docs'], meta={f:row[f] for f in meta_cols})


In [ ]:
#| exports
@patch
def tfm(self:IOMRepoReader, df:pd.DataFrame):
    "Transform raw dataframe to evaluation objects"
    df_proc = self._proc_lists(self._proc_dates(df.copy()))
    df_proc['id'] = df_proc.apply(self._mk_id, axis=1)
    df_proc['docs'] = df_proc.apply(self._mk_docs, axis=1)
    return [self._to_eval(row) for _,row in df_proc.iterrows()]

In [ ]:
reader = IOMRepoReader(fname)
evals = reader()
evals[0]


### EVALUATION OF IOM’S MIGRATION DATA STRATEGY
**Year:** 2025 | **Organization:** IOM | **Countries:** Worldwide

**Documents:** 2 available  
**ID:** `9992310969aa2f428bc8aba29f865cf3`


Finally, `to_json` runs the full pipeline and saves the results:

In [ ]:
#| exports
@patch
def to_json(self:IOMRepoReader, out_path:Path):
    evals = self()
    evals_dict = [dict(id=e.id, docs=e.docs, meta=e.meta) for e in evals]
    with open(out_path, 'w', encoding='utf-8') as f: json.dump(evals_dict, f, indent=4, ensure_ascii=False)

In [ ]:
reader = IOMRepoReader(fname)
out_path = Path('files/test/iom_evals_test.json')
reader.to_json(out_path)
out_path.exists()

True

To use the reader:

In [ ]:
#| eval: false
reader = IOMRepoReader(fname)
evaluations = reader()

The reader produces a list of JSON objects, where each object represents an evaluation with:

- `id`: A unique MD5 hash identifier generated from specified fields
- `docs`: A list of associated documents, each containing:
  - `Document Subtype`: Type of evaluation document (e.g. report, brief)
  - `File URL`: Direct link to download the document
  - `File description`: Brief description of the document contents
- `meta`: Additional metadata about the evaluation

Then serialize as `json` for further use:

In [ ]:
#| eval: false
reader.to_json('files/test/evaluations.json')

## Utilities

In [ ]:
#| exports
default_config = AttrDict(id='id', docs='docs', url='url')

In [ ]:
#| exports
def load_evals(json_file):
    "Load evaluations from JSON file"
    return L([Evaluation(**o) for o in json.loads(Path(json_file).read_text())])

In [ ]:
#| eval: false
fname = 'files/test/evaluations.json'
evals = load_evals(fname)
evals[0]


### EVALUATION OF IOM’S MIGRATION DATA STRATEGY
**Year:** 2025 | **Organization:** IOM | **Countries:** Worldwide

**Documents:** 2 available  
**ID:** `9992310969aa2f428bc8aba29f865cf3`


## Find eval by title or url

In [ ]:
#| exports
def in_docs(
    ev:Evaluation, # Evaluation object
    url:str # URL of an evaluation report 
    ):
    "Check if a URL is in the documents of an evaluation" 
    return any(L(ev.docs).filter(lambda x: x['url'] == url))

In [ ]:
url = "https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/AAP%20Evaluation%20Report_final_.pdf"
fname = 'files/test/evaluations.json'
evals = load_evals(fname)
ev = first(evals.filter(lambda x: x.id == '6c3c2cf3fa479112967612b0baddab72'))

test_eq(in_docs(ev, url), True)
test_eq(in_docs(ev, "https://fake.url/nothere.pdf"), False)

In [ ]:
#| exports
def find_eval(
    evals:list, # List of evaluations
    query:str, # Title or URL of evaluation
    by:str='title' # 'title' or 'url'
    ): 
    "Find evaluation by title or URL"
    if by == 'title': return first([o for o in evals if o.meta['Title'] == query])
    if by == 'url': return first([o for o in evals if in_docs(o, query)])

In [ ]:
title = 'Evaluation of IOM Accountability to Affected Populations'
url = "https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/AAP%20Evaluation%20Report_final_.pdf"
test_eq(find_eval(evals, title, by='title').id, '6c3c2cf3fa479112967612b0baddab72')
test_eq(find_eval(evals, url, by='url').id, '6c3c2cf3fa479112967612b0baddab72')
test_eq(find_eval(evals, 'Nonexistent Title', by='title'), None)
test_eq(find_eval(evals, 'https://fake.url/nowhere.pdf', by='url'), None)

In [ ]:
find_eval(evals, title, by='title')


### Evaluation of IOM Accountability to Affected Populations
**Year:** 2025 | **Organization:** IOM | **Countries:** Worldwide

**Documents:** 4 available  
**ID:** `6c3c2cf3fa479112967612b0baddab72`
